API call for final project

In [ ]:
!pip install mysql-connector-python
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd
import asyncio
import time
import mysql.connector

Making the API call

In [ ]:
!pip install mysql-connector-python
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd
import asyncio
import time
import mysql.connector

columns = ["stock", "company", "grossprofits", "targetmeanprice", "totalCash"]
df = pd.DataFrame(columns=columns)

async def wait_60_seconds():
    print("Waiting for 60 seconds...")
    await asyncio.sleep(60)
    print("60 seconds have passed. Now executing the code.")

apikey = "8flfzPPPtT44X5HahwLq58kJtm7Pp6PP3a8GimV5"
url = "https://yfapi.net/v6/finance/quote"

stocks = ["NVDA", "AMD", "INTC"]
headers = {'x-api-key': apikey}

# Creating an empty DataFrame to hold the API data
api_df = pd.DataFrame(columns=columns)

for _ in range(60):  # Run the loop for 60 minutes (60 * 60 seconds)
    for stock in stocks:
        querystr = {"symbols": stock, "region": "US", "lang": "en"}

        response = requests.request("GET", url, headers=headers, params=querystr)
        stock_json = response.json()

        if ("quoteResponse" in stock_json
                and "result" in stock_json["quoteResponse"]
                and stock_json["quoteResponse"]["result"][0].get("triggerable") == False
        ):
            print(f"Error: Stock ticker or information does not exist for {stock}")
        else:
            print(stock_json)

        url2 = "https://yfapi.net/v11/finance/quoteSummary/" + stock
        querystr2 = {"region": "US", "lang": "en", "modules": "financialData"}
        response2 = requests.request("GET", url2, headers=headers, params=querystr2)
        stock_json2 = response2.json()

        if ("quoteResponse" in stock_json2
                and "result" in stock_json2["quoteResponse"]
                and stock_json2["quoteResponse"]["result"][0].get("triggerable") == False
        ):
            print(f"Error: Stock ticker or information does not exist for {stock}")
        else:
            print(stock_json2)

        name = str(stock_json["quoteResponse"]['result'][0]["longName"])
        price = str(stock_json['quoteResponse']['result'][0]['regularMarketPrice'])
        grossprofits = str(stock_json2["quoteSummary"]['result'][0]['financialData']['grossProfits']['fmt'])
        targetmeanprice = str(stock_json2["quoteSummary"]['result'][0]['financialData']['targetMeanPrice']['fmt'])
        totalcash = str(stock_json2["quoteSummary"]['result'][0]['financialData']['totalCash']['fmt'])

        print(f"Stock: {stock} Company: {name} Price: {price} Gross Profits: {grossprofits}")

        # Append API data to the DataFrame
        api_df = api_df.append({
            "stock": stock,
            "company": name,
            "price":price,
            "grossprofits": grossprofits,
            "targetmeanprice": targetmeanprice,
            "totalCash": totalcash
        }, ignore_index=True)

    if _ != 59:  # Don't wait after the last iteration
        print("Waiting for 60 seconds before next API call...")
        time.sleep(60)  # Wait for 60 seconds before the next API call

# Insert all data into the database after the loop completes
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "Uva!1819"
dst_dbname = "project2_db"
mydb = mysql.connector.connect(
    host=host_ip,
    user=user_id,
    passwd=pwd,
    database=dst_dbname)
table_name = 'stock' 

mycursor = mydb.cursor()
for index, row in api_df.iterrows():
    sql = "INSERT INTO " + table_name + " (stock, price, company, grossprofits, targetmeanprice, totalCash) VALUES (%s, %s, %s, %s, %s, %s)"
    val = (row['stock'], row['price'], row['company'], row['grossprofits'], row['targetmeanprice'], row['totalCash'])
    mycursor.execute(sql, val)
    mydb.commit()

    
## Calculate average price and gross profits for each stock
avg_data = api_df.groupby('stock')[['regularMarketPrice', 'grossprofits']].mean().reset_index()
print(avg_data)
median_grossprofits = df['grossprofits'].median()
print(median_grossprofits)
std_price = df['regularMarketPrice'].std()
print(std_price)